In [12]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta
import time
import os

import warnings
warnings.filterwarnings('ignore')

In [14]:
#A function get url, driver ,the information on reservation time and df and returns an updated df with the 100 additional hotels.
Unsuccessful_list  = [] # A list that saves the names of the hotels that have a problem in order to optimize and not try to extract information again
def get_hotels(url , driver ,booking_df,Snapshot_date,checking_date_type,checkout_date_type):
    driver.get(url)
    time.sleep(1)
    hotel_name_list =[]
    page_number=1
    index = 1
    while(len(hotel_name_list) <100 and page_number<=6): #take 100 hotel 
        time.sleep(2)
        text_boxes_list = driver.find_elements(by= By.CLASS_NAME , value='b978843432')
        for t_box in text_boxes_list:# run on full page
            if (len(hotel_name_list) >= 100):
                break
            try:
                hotel = {'Snapshot' : None ,'Index':None,'Hotel Name':None,'TTT':None,'LOS':None ,"Grade":None,
                         "Num of Reviews":None,"Curr Price":None,"Original Price":None,"Distance from center":None,
                         "Type of room":None,"Location grade":None ,"Extra Data":None}
                name = t_box.find_element(by= By.CLASS_NAME , value="e13098a59f").text
                name = name.split('\n')[0]
                if(name in hotel_name_list):#Avoid hotels that are advertised several times
                    continue   
                if (name in Unsuccessful_list):#Avoid hotels that we cannot extract
                    continue
                hotel['Hotel Name'] = (name)
                hotel['Snapshot'] = Snapshot_date
                delta_TTT =  checking_date_type - Snapshot_date
                hotel['TTT'] = delta_TTT.days
                delta_LOS = checkout_date_type - checking_date_type
                hotel['LOS'] = delta_LOS.days
                score_detail = t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="review-score"]').text
                score_detail_list = score_detail.split('\n')
                hotel["Grade"] = (score_detail_list[0])
                hotel["Num of Reviews"] = (score_detail_list[2])
                hotel["Curr Price"] = (t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="price-and-discounted-price"]').text)
                hotel["Index"] = index
                hotel["Extra Data"] = t_box.find_element(by= By.CLASS_NAME , value="d22a7c133b").text
                hotel["Distance from center"] =  t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="distance"]').text
                hotel["Type of room"] = t_box.find_element(by= By.CLASS_NAME , value="df597226dd").text
                
                try:
                    hotel["Location grade"] = t_box.find_element(by= By.CLASS_NAME , value="f9afbb0024").text 
                except:
                    hotel["Location grade"] = None
            
                try:
                    hotel["Original Price"] = t_box.find_element(by =By.XPATH,value ='.//*[@class="c5888af24f e729ed5ab6"]').text     
                except:
                    hotel["Original Price"] = hotel["Curr Price"]
                
                booking_df = booking_df.append(hotel, ignore_index=True)#add the new hotel
                hotel_name_list.append(name) #add the new hotel name
                index = index +1
            except:
                print("error " + name)
                Unsuccessful_list.append(name)
        # change page
        if( len(hotel_name_list)< 100): 
            button=driver.find_element(by =By.CSS_SELECTOR,value =f'[aria-label=" {page_number+1}"]')
            time.sleep(1.5)
            button.click() 
            page_number = page_number+1
    
    
    return booking_df
    

In [15]:
#Allows us to run on different computers without a problem

path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': r"C:\Users\yarin\chromedriver.exe" , 'yair':'C:\Windows\chromdriver.exe' }
driver = webdriver.Chrome(path['yarin'])

In [16]:
# Get today's date
now = datetime.now()
Snapshot_date =now
# Get end date 35 days from now
end_date = now + timedelta(days=35)

# Create a list of all dates between today and end date
date_list = []
while now <= end_date:
    date_list.append(now.strftime('%Y-%m-%d'))
    now += timedelta(days=1)


#Creates a list whose every member is a list of checking and checkout time

combinations_list =[]
for i in range(1,len(date_list)-5):
    for j in range(1,6):
        combinations = [date_list[i],date_list[i+j]]
        combinations_list.append(combinations)
        
print(combinations_list , len(combinations_list))        
    

[['2023-03-09', '2023-03-10'], ['2023-03-09', '2023-03-11'], ['2023-03-09', '2023-03-12'], ['2023-03-09', '2023-03-13'], ['2023-03-09', '2023-03-14'], ['2023-03-10', '2023-03-11'], ['2023-03-10', '2023-03-12'], ['2023-03-10', '2023-03-13'], ['2023-03-10', '2023-03-14'], ['2023-03-10', '2023-03-15'], ['2023-03-11', '2023-03-12'], ['2023-03-11', '2023-03-13'], ['2023-03-11', '2023-03-14'], ['2023-03-11', '2023-03-15'], ['2023-03-11', '2023-03-16'], ['2023-03-12', '2023-03-13'], ['2023-03-12', '2023-03-14'], ['2023-03-12', '2023-03-15'], ['2023-03-12', '2023-03-16'], ['2023-03-12', '2023-03-17'], ['2023-03-13', '2023-03-14'], ['2023-03-13', '2023-03-15'], ['2023-03-13', '2023-03-16'], ['2023-03-13', '2023-03-17'], ['2023-03-13', '2023-03-18'], ['2023-03-14', '2023-03-15'], ['2023-03-14', '2023-03-16'], ['2023-03-14', '2023-03-17'], ['2023-03-14', '2023-03-18'], ['2023-03-14', '2023-03-19'], ['2023-03-15', '2023-03-16'], ['2023-03-15', '2023-03-17'], ['2023-03-15', '2023-03-18'], ['2023-03

In [17]:
#for dev 
##combinations_list = [combinations_list[0]]
#end dev
city = "new york"
numOfAdults=2
numOfChild=0
numOfRooms=1
booking_df = pd.DataFrame(columns=['Snapshot','Index','Hotel Name','TTT' ,'LOS' ,"Grade","Num of Reviews","Curr Price","Original Price",
                                   "Distance from center","Type of room","Location grade", "Extra Data"])  

Unsuccessful_list = []
for comb in combinations_list:#run on the combinations list
    #get the information on reservation time 
    checking_date_type = datetime.strptime(comb[0], '%Y-%m-%d')
    checkout_date_type = datetime.strptime(comb[1], '%Y-%m-%d')
    checking_date = comb[0].split('-')
    checkout_date = comb[1].split('-')
    checking_year = checking_date[0]
    checking_month = checking_date[1]
    checking_monthday =checking_date[2]
    checkout_year= checkout_date[0]
    checkout_month= checkout_date[1]
    checkout_monthday= checkout_date[2]
    url = f"https://www.booking.com/searchresults.html?ss={city}&checkin_year={checking_year}&checkin_month={checking_month}&checkin_monthday={checking_monthday}&checkout_year={checkout_year}&checkout_month={checkout_month}&checkout_monthday={checkout_monthday}&group_adults={numOfAdults}&group_children={numOfChild}&no_rooms={numOfRooms}"
    booking_df = get_hotels(url, driver , booking_df, Snapshot_date,checking_date_type,checkout_date_type)
    print(booking_df.shape)
    

error The FIDI Hotel
error The Hotel Chelsea
(100, 13)
error The Home Hotel Powered by LuxUrban
error LPWELLNESS
(200, 13)
(300, 13)
(400, 13)
error Apollo Hideaway
(500, 13)
error Club Quarters Hotel Times Square, New York
error Club Quarters Hotel Times Square, New York
error Club Quarters Hotel Times Square, New York
error Club Quarters Hotel Times Square, New York
error Club Quarters Hotel Times Square, New York
error Arlo NoMad
error Radio Hotel
error Radio Hotel
error Radio Hotel
error Radio Hotel
error Radio Hotel
error Radio Hotel
error Radio Hotel
error Radio Hotel
error Radio Hotel
error Moderne Hotel
error Moderne Hotel
error Moderne Hotel
error Moderne Hotel
error The Dominick Hotel
error The Dominick Hotel
error The Dominick Hotel
error WestHouse Hotel New York
error Pod 39
(600, 13)
(700, 13)
(800, 13)
(900, 13)
error MUSEUM MILE by CENTRAL PARK
(1000, 13)
(1100, 13)
(1200, 13)
(1300, 13)
(1400, 13)
(1500, 13)
(1600, 13)
(1700, 13)
(1800, 13)
(1900, 13)
(2000, 13)
(2100, 

In [18]:
booking_df.head(6500)

,Snapshot,Index,Hotel Name,TTT,LOS,Grade,Num of Reviews,Curr Price,Original Price,Distance from center,Type of room,Location grade,Extra Data
0,2023-03-08 10:57:52.355464,1,Pestana CR7 Times Square,0,1,8.1,"1,365 reviews",₪ 645,"₪ 1,076",1.6 km from center,Comfort Room,None,Comfort Room\n1 queen bed\nFREE cancellation •...
1,2023-03-08 10:57:52.355464,2,The Knickerbocker,0,1,8.4,"1,335 reviews","₪ 1,077","₪ 1,268",1.4 km from center,Superior Room with One King Bed,Location 9.4,Superior Room with One King Bed\n1 king bed
2,2023-03-08 10:57:52.355464,3,Margaritaville Resort Times Square,0,1,8.8,"1,384 reviews",₪ 851,₪ 851,1.5 km from center,Queen Room - Disability Access,Location 9.4,Queen Room - Disability Access\n1 queen bed\nO...
3,2023-03-08 10:57:52.355464,4,Ramada by Wyndham New York Times Square West,0,1,6.9,"2,574 reviews",₪ 461,₪ 461,1.2 km from center,"1 King Bed, Lower Level Non-Smoking",None,"1 King Bed, Lower Level Non-Smoking\n1 king be..."
4,2023-03-08 10:57:52.355464,5,Royalton New York,0,1,7.6,"1,380 reviews",₪ 902,₪ 902,1.4 km from center,Standard Queen Room,None,Standard Queen Room\n1 queen bed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6495,2023-03-08 10:57:52.355464,96,La Quinta by Wyndham Time Square South,12,5,8.1,"2,080 reviews","₪ 4,585","₪ 4,585",1.7 km from center,King Room - Non-Smoking,None,King Room - Non-Smoking\n1 king bed\nBreakfast...
6496,2023-03-08 10:57:52.355464,97,Placemakr Wall Street,12,5,8.7,"1,016 reviews","₪ 3,806","₪ 3,806",7.3 km from center,Studio,None,Studio\nEntire studio • 1 bathroom • 30m²\n1 t...
6497,2023-03-08 10:57:52.355464,98,HGU New York,12,5,7.4,"1,540 reviews","₪ 4,209","₪ 4,209",2.4 km from center,King Standard,None,King Standard\n1 king bed\nFree cancellation\n...
6498,2023-03-08 10:57:52.355464,99,Hampton Inn Manhattan Chelsea,12,5,7.8,"1,436 reviews","₪ 4,614","₪ 4,614",2.9 km from center,King Room - Mobility Accessible/Non-Smoking,None,King Room - Mobility Accessible/Non-Smoking\n1...


In [19]:
booking_df.shape

(15000, 13)

In [20]:
booking_df.groupby(['TTT','LOS']).count()

Snapshot  Index  Hotel Name  Grade  Num of Reviews  Curr Price  \
TTT LOS                                                                   
0   1         100    100         100    100             100         100   
    2         100    100         100    100             100         100   
    3         100    100         100    100             100         100   
    4         100    100         100    100             100         100   
    5         100    100         100    100             100         100   
...           ...    ...         ...    ...             ...         ...   
29  1         100    100         100    100             100         100   
    2         100    100         100    100             100         100   
    3         100    100         100    100             100         100   
    4         100    100         100    100             100         100   
    5         100    100         100    100             100         100   

         Original Price  Distance from center  Type of room  Location grade  \
TTT LOS                                                                       
0   1               100                   100           100              49   
    2               100                   100           100              46   
    3               100                   100           100              46   
    4               100                   100           100              46   
    5               100                   100           100              45   
...                 ...                   ...           ...             ...   
29  1               100                   100           100              48   
    2               100                   100           100              44   
    3               100                   100           100              48   
    4               100                   100           100              44   
    5               100                   100           100              43   

         Extra Data  
TTT LOS              
0   1           100  
    2           100  
    3           100  
    4           100  
    5           100  
...             ...  
29  1           100  
    2           100  
    3           100  
    4           100  
    5           100  

[150 rows x 11 columns]

In [21]:
driver.quit()

save the df in csv

In [25]:
file_name = "Booking_hotels.csv"
if os.path.exists(file_name):#if the "Booking_hotels.csv" exists
    print("exists")
    booking_df.to_csv(file_name,mode = 'a' , index= False ,header=False)#append without the header
else:
    print("new")
    booking_df.to_csv(file_name,mode = 'a' , index= False ,header=True)#append with the header

exists
